# **Übung 8, Teil 1: Numerische Lösung der Wellengleichung erster Ordnung**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Erhöhen der Plot-Auflösung
plt.rcParams["figure.dpi"] = 140

XLIM = [0, 1]
YLIM1 = [-1.1, 1.1]
YLIM2 = [-0.5, 1.5]

### **8.1.1 Numerische Parameter**

In [2]:
# Konstant
m = 50              # Anzahl räumlicher Stützstellen
c = 1               # Ausbreitungsgeschwindigkeit

# 🟠

In [3]:
# Dieser Parameter kann verändert werden
dt = 0.02          # Zeitschrittweite

In [ ]:
total_time = 1.2 / abs(c)     # Simulationszeit
dx = 1 / m          # räumliche Schrittweite (auf 1m)
n = int(total_time / dt)

print("Räumlich:    ", m, "Stützstellen pro Meter ergibt eine räumliche Schritteweite von", "%.2f" % dx, "m")
print("Zeitlich:    ", dt, "s Zeitschrittweite über", total_time, "s ergibt", n, "Zeitschritte")

### **8.1.2 Gitterpunkte und Anfangslösung erzeugen**

In [5]:
# Funktion zum setzten der Anfangsbedingung (Testfunktion)
def initial_cond(x):
    return np.cos(4 * np.pi * x) 

In [6]:
# Gitterpunkte 
x = np.linspace(0, 1, m + 1)   

# Lösungsmatrix für phi initialisieren:
#   - die räumliche Ausbreitung zum Zeitpunkt t ist in Spalte t untergebracht phi[:, t]
#   - die zeitliche Ausbreitung am Ort x ist in Reihe x untergebracht phi[x, :]
phi = np.zeros((m + 1, n + 1))   

# Anfangsbedingung
phi[:, 0] = initial_cond(x)

In [ ]:
# Visualisierung der Anfangslösung
plt.figure(figsize=(6, 3))
plt.plot(x, phi[:, 0], color="gray")
plt.title('Anfangslösung der Wellengleichung')
plt.xlabel('x')
plt.ylabel(r'$\phi(x, t=0)$')
plt.grid(True)
plt.xlim(XLIM)
plt.ylim(YLIM1)
plt.tight_layout()
plt.show()

### **8.1.3 Löser Schleife**

In [8]:
# Funktion zum setzten der Randbedingungen
def apply_BC(phi, j):
    if c < 0:
        phi[0, j] = phi[1, j - 1]  
        phi[-1, j] = 1 
    else:
        phi[-1, j] = phi[-2, j - 1]
        phi[0, j] = 1    
    return phi


# Löser-Schleife
def simple_solver(phi, dt, c, dx, n, m):
    """ 
    Zeitlich:   Einstufiges, explizites Zeitschrittverfahren (1. Ordnung) 
    Räumlich:   Aufwind-Schema (Rückwärtsschema 1. Ordnung)
    """
    for j in range(1, n + 1):
        phi = apply_BC(phi, j)
        for i in range(1, m):
            phi[i, j] = phi[i, j-1] - dt * (c * (phi[i, j-1] - phi[i - 1, j-1]) / dx)
            
    return phi

In [9]:
# Löser-Schleife
phi = simple_solver(phi, dt, c, dx, n, m)

# Animation der Lösung
def animation_phi(title, YLIM, phi=phi):
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.set_xlim(XLIM)
    ax.set_ylim(YLIM)
    ax.set_title(f"{title}, t = 0")
    ax.plot(x, phi[:, 0], lw=2, color="gray", label='Anfangslösung')
    line, = ax.plot([], [], lw=2, color="firebrick", label='Lösung')
    ax.set_xlabel('x')
    ax.set_ylabel(r'$\phi(x, t)$')
    ax.grid(True)
    ax.legend(loc="lower center")
    plt.tight_layout()

    def animate(j):
        line.set_data(x, phi[:, j])
        ax.set_title(f"{title}, t = {j*dt:.2f} s")
        return line,

    ani = animation.FuncAnimation(fig, animate, frames=n, interval=50, blit=True)
    plt.close(fig)
    return ani

In [ ]:
ani = animation_phi("Lösung der Wellengleichung", YLIM=YLIM1)
HTML(ani.to_jshtml())

## **Aufgabe 1 a) Berechnungsvorschrift für zulässige Zeitschrittweite**

Ergänzen Sie das Programm um eine Berechnungsvorschrift für die zulässige Zeitschrittweite. Überprüfen Sie die Zeitschrittweitenberechnung durch Variation der relevanten Simulationsparameter (z.B. Ausbreitungsgeschwindigkeit c). Führen Sie anschließend die CFL-Zahl als frei wählbaren Parameter `clf` ein und stellen Sie ihren Einfluss auf die Lösung dar.

# 🟠

In [11]:
# Ausbreitungsgeschwindigkeit variieren
c = 5     

In [12]:
# Berechnungsvorschrift für dt
#    - der Zeitschritt muss limitiert werden, damit die Welle in der Zeit dt einen Weg <= dx zurücklegt
#    - für größere Werte von dt breitet sich Information räumlich schneller aus als sie zeitlich abgebildet werden kann
def get_dt():
    dt = cfl * dx / abs(c)
    return dt

cfl = 1

In [ ]:
dt = get_dt()
total_time = 1.2 / abs(c) 
n = int(total_time / dt)

print("Räumlich:    ", m, "Stützstellen pro Meter ergibt eine räumliche Schritteweite von", "%.2f" % dx, "m")
print("Zeitlich:    ", dt, "s Zeitschrittweite über", total_time, "s ergibt", n, "Zeitschritte")

In [ ]:
# phi neu initialisieren
phi = np.zeros((m + 1, n + 1))   

# AB
phi[:, 0] = initial_cond(x)

# Löser-Schleife
phi = simple_solver(phi, dt, c, dx, n, m)

ani = animation_phi("Lösung mit Berechnungsvorschrift", YLIM=YLIM1)
HTML(ani.to_jshtml())

Die Berechnungsvorschrift `dt = dx / abs(c)` erzeugt für alle positiven Werte von c eine stabile Lösung. Man kann darauf basierend die CFL Zahl definieren:
\begin{equation}
CFL = c * \frac{\Delta t}{\Delta x}
\end{equation}
Damit ändert sich die Berechnungsvorschrift zu `dt = cfl * dx / abs(c)`.

In [ ]:
# Konstante Parameter setzen
cfl_study = [0.8, 1.0, 1.2]
c = 1 
total_time = 0.5 / abs(c) 

# Abbildung mit drei plots untereinander initialisieren
fig, axes = plt.subplots(3, 1, figsize=(6, 8), sharex=True)


for k, cfl in enumerate(cfl_study):
    # Zeitschritt(-weite)
    dt = get_dt()
    n = int(total_time / dt)

    # Lösungsmatrix mit AB initialisieren
    phi = np.zeros((m + 1, n + 1))   
    phi[:, 0] = initial_cond(x)

    # Löser-Schleife:
    phi = simple_solver(phi, dt, c, dx, n, m)

    # Ergebnis plotten
    axes[k].plot(x, phi[:, 0], lw=2, color="gray", label='Anfangslösung')
    axes[k].plot(x, phi[:, -1], lw=2, color="firebrick", label='Lösung')
    axes[k].set_title("CFL = {}".format(cfl))
    axes[k].set_xlim(XLIM)
    axes[k].set_xlabel('x')
    axes[k].set_ylabel(r'$\phi(x, t)$')
    axes[k].grid(True)
    axes[k].legend(loc="lower center")
plt.tight_layout()

## **Aufgabe 1 b) Negative Ausbreitungsrichtung**

Kehren Sie Ausbreitungsrichtung in der Wellengleichung um und ändern Sie die räumliche Diskretisierung so, dass Sie für beide Ausbreitungsrichtungen stabil ist. Demonstrieren Sie dies durch geeignete Tests.


# 🟠

In [16]:
# Ausbreitungsgeschwidnigkeit varriieren (negativ/ positiv)
c = -1

In [17]:
# Löser-Schleife anpassen
def simple_solver_extended(phi, dt, c, dx, n, m):
    """ 
    Zeitlich:   Einstufiges, explizites Zeitschrittverfahren (1. Ordnung) 
    Räumlich:   Aufwind-Schema (Vorwärts- und Rückwärtsschema 1. Ordnung)
    """
    for j in range(1, n+1):
        phi = apply_BC(phi, j)
        for i in range(1, m):
            if c < 0:
                phi[i, j] = phi[i, j-1] - dt * (c * (phi[i+1, j-1] - phi[i, j-1]) / dx)
            else:
                phi[i, j] = phi[i, j-1] - dt * (c * (phi[i, j-1] - phi[i - 1, j-1]) / dx)
            
    return phi

In [ ]:
# Zeitschritt(-weite) neu berechnen
cfl = 1
dt = get_dt()
total_time = 1.2 / abs(c) 
n = int(total_time / dt)

# Lösungsmatrix mit AB erzeugen
phi = np.zeros((m + 1, n + 1)) 
phi[:, 0] = initial_cond(x)

# Neue Implementierung der Löser-Schleife
phi = simple_solver_extended(phi, dt, c, dx, n, m)

ani = animation_phi("Lösung mit negativer Ausbreitungsgeschwindigkeit", YLIM=YLIM1)
HTML(ani.to_jshtml())

## **Aufgabe 1 c) Einfluss der räumlichen Diskretisierung bei verschiedenen Schemata**

Das nachfolgende Skript verwendet ein Aufwind-Schema sowie ein zentrales Schema mit 4. Differenzen in Verbindung mit einem dreistufigen Runge-Kutta Zeitschrittverfahren. Variieren Sie systematisch die Anzahl der Stützstellen `m`, um den Einfluss der räumlichen Auflösung auf die Ergebnisse für beide Diskretisierungsschemata darzustellen.


In [19]:
# Funktion zum setzten der Randbedingungen für Runge-Kutta Verfahren
def apply_BC_RK(phi):
    if c < 0:
        phi[0] = 2 * phi[1] - phi[2]
        phi[-1] = 1 
    else:
        phi[0] = 1 
        phi[-1] = 2 * phi[-2] - phi[-3]   
    return phi


def three_stage_RK_upwind(phi, a1, a2, dt, c, dx, n, m):
    """ 
    Zeitlich:   Dreistufiges Runge-Kutta Verfahren (1. Ordnung)
    Räumlich:   Aufwind-Schema (1. Ordnung)
    """
    for j in range(1, n+1):
        # Stufe 1
        phi = apply_BC(phi, j)
        for i in range(1, m):
            if c<0:
                a1[i, 0] = phi[i, j-1] - 0.15 * dt * (c * (phi[i+1, j-1] - phi[i, j-1]) / dx)
            else:
                a1[i, 0] = phi[i, j-1] - 0.15 * dt * (c * (phi[i, j-1] - phi[i - 1, j-1]) / dx)

        # Stufe 2
        a1 = apply_BC_RK(a1)
        for i in range(1, m):
            if c<0:
                a2[i, 0] = phi[i, j-1] - 0.5 * dt * (c * (a1[i+1, 0] - a1[i, 0]) / dx)
            else:
                a2[i, 0] = phi[i, j-1] - 0.5 * dt * (c * (a1[i, 0] - a1[i-1, 0]) / dx)
                
        # Stufe 3
        a2 = apply_BC_RK(a2)
        for i in range(1, m):
            if c<0:
                phi[i, j] = phi[i, j-1] - 1.0 * dt * (c * (a2[i+1, 0] - a2[i, 0]) / dx)
            else:
                phi[i, j] = phi[i, j-1] - 1.0 * dt * (c * (a2[i, 0] - a2[i-1, 0]) / dx)
        
    return phi


def d4(u, i, j, c, m):
    """ Berechnung von vierten Differenzen """
    
    u_iplus2 = 2 * u[i+1, j] - u[i, j] if i == m - 1 else u[i+2, j]
    u_iminus2 = 2 * u[i-1, j] - u[i, j] if i == 1 else u[i-2, j]

    # Berechnung der vierten Differenzen 
    return (abs(c) * (u_iplus2 - 3 * u[i+1, j] + 3 * u[i, j] - u[i-1, j]) - abs(c) * (u[i+1, j] - 3 * u[i, j] + 3 * u[i-1, j] - u_iminus2)) / dx


def three_stage_RK_central_diff(phi, a1, a2, dt, c, dx, n, m, k4):
    """ 
    Zeitlich:   Dreistufiges Runge-Kutta Verfahren (1. Ordnung)
    Räumlich:   Zentrales Schema mit vierten Differenzen (2. Ordnung)
    """
    for j in range(1, n+1):
        # Stufe 1
        phi = apply_BC(phi, j)
        for i in range(1, m):
            a1[i, 0] = phi[i, j-1] - 0.15 * dt * (c * (phi[i+1, j-1] - phi[i-1, j-1]) / 2 / dx + k4 * d4(phi, i, j-1, c, m))

        # Stufe 2
        a1 = apply_BC_RK(a1)
        for i in range(1, m):
            a2[i, 0] = phi[i, j-1] - 0.5 * dt * (c * (a1[i+1, 0] - a1[i-1, 0]) / 2 / dx + k4 * d4(a1, i, 0, c, m))
                
        # Stufe 3
        a2 = apply_BC_RK(a2)
        for i in range(1, m):
            phi[i, j] = phi[i, j-1] - 1.0 * dt * (c * (a2[i+1, 0] - a2[i-1, 0]) / 2 / dx + k4 * d4(a2, i, 0, c, m))
        
    return phi

# 🟠

In [20]:
# Anzahl der Stützstellen variieren
m = 50

In [21]:
# Konstanten
c = 0.1
cfl = 1
k4 = 1 / 32

# Simulationsparameter neu berechnen
x = np.linspace(0, 1, m + 1)  
dx = 1 / m
dt = get_dt()
total_time = 0.5 / abs(c) 
n = int(total_time / dt)

In [22]:
# Aufwind-Schema initialisieren
phi_upwind = np.zeros((m + 1, n + 1))
phi_upwind[:, 0] = initial_cond(x)
a1 = np.zeros((m + 1, 1))
a2 = np.zeros((m + 1, 1))

# Lösung mit Aufwind-Schema
phi_upwind = three_stage_RK_upwind(phi_upwind, a1, a2, dt, c, dx, n, m)

# Zentrale-Differenzen-Schema initialisieren
phi_central_diff = np.zeros((m + 1, n + 1))
phi_central_diff[:, 0] = initial_cond(x)
a1 = np.zeros((m + 1, 1))
a2 = np.zeros((m + 1, 1))

# Lösung mit zentralem Schema
phi_central_diff = three_stage_RK_central_diff(phi_central_diff, a1, a2, dt, c, dx, n, m, k4)

In [ ]:
# Ergebnisse plotten
fig, axes = plt.subplots(1, 2, figsize=(12, 3), sharey=True)

for i, ax in enumerate(axes):
    title_pre = "Aufwind-" if i == 0 else "Zentrales "
    phi = phi_upwind if i == 0 else phi_central_diff

    ax.plot(x, phi[:, 0], lw=2, color="gray", label='Anfangslösung')
    ax.plot(x, phi[:, -1], lw=2, color="firebrick", label='Lösung')
    ax.set_title(title_pre + "Schema" + f" (m = {m})")
    ax.set_xlim(XLIM)
    ax.set_xlabel('x')
    ax.set_ylabel(r'$\phi(x, t)$')
    ax.grid(True)
    ax.legend(loc="lower center")

## **Aufgabe 1 d) Implementierung künstlicher Dissipation**

In dieser Teilaufgabe wird eine neue Testfunktion in Form einer Stufe mit Sprung von 0 auf 1 verwendet. Wenden Sie die Verfahren auf die neue Anfangslösung mit Diskontinuität an. Ergänzen Sie das zentrale Schema um eine geeignete künstliche Dissipation und dokumentieren Sie ihre stabilisierende Wirkung.



In [24]:
# Funktion zum Setzen der Anfangslösung mit Diskontinuität
def initial_cond(x):
    phi_0 = np.ones_like(x)
    phi_0[x < 0.2] = 0  

    return phi_0


# Funktion zum Setzen der Randbedingungen
def apply_BC(phi, j):
    if c < 0:
        phi[0, j] = phi[1, j - 1]  
        phi[-1, j] = 1 
    else:
        phi[0, j] = 0           # theoretisch überflüssig, da phi als Matrix mit Werten 0 initialisiert wird
        phi[-1, j] = phi[-2, j - 1] 
    return phi


# Funktion zum Setzen der Randbedingungen für Runge-Kutta Verfahren
def apply_BC_RK(phi):
    if c < 0:
        phi[0] = 2 * phi[1] - phi[2]
        phi[-1] = 1 
    else:
        phi[0] = 0              # theoretisch überflüssig, da phi als Matrix mit Werten 0 initialisiert wird
        phi[-1] = 2 * phi[-2] - phi[-3]  
    return phi


def d2(u, i, j, c):
    """ Berechnung von zweiten Differenzen (Dissipationstermen)"""
    return abs(c) * (u[i+1, j] - 2 * u[i, j] + u[i-1, j]) / dx


def three_stage_RK_central_diff_dissipation(phi, a1, a2, dt, c, dx, n, m, k2):
    """ 
    Zeitlich:   Dreistufiges Runge-Kutta Verfahren (1. Ordnung)
    Räumlich:   Zentrales Schema mit vierten Differenzen (2. Ordnung)
    """
    for j in range(1, n+1):
        # Stufe 1
        phi = apply_BC(phi, j)
        for i in range(1, m):
            a1[i, 0] = phi[i, j-1] - 0.15 * dt * (c * (phi[i+1, j-1] - phi[i-1, j-1]) / 2 / dx - k2 * d2(phi, i, j-1, c))

        # Stufe 2
        a1 = apply_BC_RK(a1)
        for i in range(1, m):
            a2[i, 0] = phi[i, j-1] - 0.5 * dt * (c * (a1[i+1, 0] - a1[i-1, 0]) / 2 / dx - k2 * d2(a1, i, 0, c))
                
        # Stufe 3
        a2 = apply_BC_RK(a2)
        for i in range(1, m):
            phi[i, j] = phi[i, j-1] - 1.0 * dt * (c * (a2[i+1, 0] - a2[i-1, 0]) / 2 / dx - k2 * d2(a2, i, 0, c))
        
    return phi

In [ ]:
# Visualisierung der Anfangslösung
plt.figure(figsize=(6, 3))
plt.plot(x, initial_cond(x), color="gray")
plt.title('Neue Testfunktion mit Sprung')
plt.xlabel('x')
plt.ylabel(r'$\phi(x, t=0)$')
plt.grid(True)
plt.xlim(XLIM)
plt.ylim(YLIM2)
plt.tight_layout()
plt.show()

In [26]:
# Konstanten
m = 200
c = 0.1
cfl = 1
k2 = 0.5      # Skalierungsterm

# Simulationsparameter neu berechnen
x = np.linspace(0, 1, m + 1)  
dx = 1 / m
dt = get_dt()
total_time = 1 / abs(c) 
n = int(total_time / dt)

# Aufwind-Schema initialisieren
phi_upwind = np.zeros((m + 1, n + 1))
phi_upwind[:, 0] = initial_cond(x)
a1 = np.zeros((m + 1, 1))
a2 = np.zeros((m + 1, 1))

# Lösung mit Aufwind-Schema
phi_upwind = three_stage_RK_upwind(phi_upwind, a1, a2, dt, c, dx, n, m)

# Zentrale-Differenzen-Schema initialisieren
phi_central_diff = np.zeros((m + 1, n + 1))
phi_central_diff[:, 0] = initial_cond(x)
a1 = np.zeros((m + 1, 1))
a2 = np.zeros((m + 1, 1))

# Lösung mit zentralem Schema
phi_central_diff = three_stage_RK_central_diff_dissipation(phi_central_diff, a1, a2, dt, c, dx, n, m, k2)

In [ ]:
ani = animation_phi("Diskontinuität gelöst mit Aufwind-Schema", YLIM=YLIM2, phi=phi_upwind)
HTML(ani.to_jshtml())

In [ ]:
ani = animation_phi("Diskontinuität gelöst mit Zentralem Schema", YLIM=YLIM2, phi=phi_central_diff)
HTML(ani.to_jshtml())